In [2]:
import pandas
import mido
import os
import re
import shutil
from multiprocessing import Pool, Value, Array
import glob
import ntpath
import time


In [3]:
# Use this to strip titles and instrument data of non-alphabetic characters
pattern = "[^a-zA-Z]+"

In [4]:
import os
os.getcwd()


'/home/branchedelac/code/MarkBerkovics/BandIt/notebooks'

In [5]:
class FileProcessor():
    def __init__(self, output_path):
        self.saved_songs = []
        self.errors = 0
        self.num_processed = 0
        self.output_path = output_path

    def parse_file(self, midi_path):
        try:
            midi_file = mido.MidiFile(midi_path, clip=True)
            dg_song = self.identify_drum_and_guitar(midi_file)
            if dg_song:
                location_, filename = ntpath.split(midi_path)
                midi_file.save(f'{self.output_path}/{filename}')
                #shutil.copyfile(midi_file, f'{self.output_path}/{filename}')

        except Exception as ke:
            self.errors +=1


    def identify_drum_and_guitar(self, midi):
        guitars = {"guitar", "guita", "gtr", "egtr", "eguitar"}
        drums = {"drums", "drum", "drumsperc", "perc", "percussion"}

        pattern = "[^a-zA-Z ]+"
        has_drum = False
        has_guitar = False

        for track in midi:
            # Check if instruments in track name
            if track.type == "track_name":
                words = {re.sub(pattern, '', word.lower()) for word in track.name.split()}
                if words.intersection(guitars):
                    has_guitar = True
                if words.intersection(drums):
                    has_drum = True
        if has_drum and has_guitar:
            return True

        # Check if instruments in track text
        for track in midi:
            if track.type == "text":
                words = {re.sub(pattern, '', word.lower())  for word in track.text.split()}
                if words.intersection(guitars):
                    has_guitar = True
                if words.intersection(drums):
                    has_drum = True
        if has_drum and has_guitar:
            return True

In [24]:
# Get raw midi files from..
raw_data_folder = os.path.join(os.pardir, "raw_data", "Unzipped")
#raw_data_folder = os.path.join(os.pardir, "test_data")
midi_file_paths = glob.glob(f"{raw_data_folder}/*.mid") + glob.glob(f"{raw_data_folder}/*/*.mid")

# Save all midi guitar and drum files to...
output_folder = os.path.join(os.pardir, "data_deduped")
print(output_folder)

../data_deduped


In [25]:
%time
file_processor = FileProcessor(output_folder)

# Identify and save files with guitar and drum
print(f"Processing {len(midi_file_paths)} files in {raw_data_folder}...")
print(f"Saving to {output_folder}...")
pool = Pool(5)
result = pool.map(file_processor.parse_file, midi_file_paths)

# Remove duplicated
print(f"Removing duplicates from {output_folder}...")
already_seen = []
saved_files = glob.glob(f"{output_folder}/*.mid")
print(saved_files)
for midi_file in saved_files:
    title = re.sub(pattern, "", midi_file).lower()
    if title in already_seen:
        os.remove(midi_file)
    else:
        already_seen.append(title)

print("Total number of non-duplicate songs with drums and guitar saved:", len(saved_files))


CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 10 µs
Processing 4062 files in ../raw_data/Unzipped...
Saving to ../data_deduped...
Removing duplicates from ../data_deduped...
[]
Total number of non-duplicate songs with drums and guitar saved: 0


In [ ]:
print("Total number of non-duplicate songs with drums and guitar saved:", len(saved_files))
